In [1]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")


!pip install fastapi uvicorn pyngrok  ultralytics  python-multipart
# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [2]:
# Import necessary libraries
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import StreamingResponse, JSONResponse
from pyngrok import ngrok
import cv2
import numpy as np
from io import BytesIO
from PIL import Image
import asyncio
from ultralytics import YOLO
import base64

# Apply nest_asyncio to allow running FastAPI in Jupyter
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Load YOLOv11 model (choose appropriate model variant: yolov11n, yolov11s, etc.)


# Load a model
model = YOLO("yolo11n.pt")


# Health check endpoint
@app.get("/health")
async def health_check():
    return JSONResponse(content={"status": "healthy"}, status_code=200)

# Define the image processing function with YOLOv11
def process_image(image: np.ndarray, prompt: str) -> tuple:
    # Perform YOLOv11 inference
    results = model(image)

    # Extract detection results
    detections = results[0].boxes  # Get bounding boxes
    bounding_boxes = detections.xyxy.cpu().numpy().tolist()  # [x1, y1, x2, y2]
    confidences = detections.conf.cpu().numpy().tolist()  # Confidence scores
    class_ids = detections.cls.cpu().numpy().astype(int).tolist()  # Class IDs

    # Get class names
    class_names = [model.names[cls_id] for cls_id in class_ids]

    # Draw bounding boxes on the image
    processed_image = image.copy()
    for box, conf, cls_name in zip(bounding_boxes, confidences, class_names):
        x1, y1, x2, y2 = map(int, box)
        # Draw rectangle
        cv2.rectangle(processed_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Add label
        label = f"{cls_name} {conf:.2f}"
        cv2.putText(processed_image, label, (x1, y1 - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return processed_image, bounding_boxes, class_names, confidences

# FastAPI endpoint to handle image upload and prompt
@app.post("/process-image/")
async def process_image_endpoint(file: UploadFile = File(...), prompt: str = Form(...)):
    try:
        # Read the uploaded image
        image_data = await file.read()
        image = Image.open(BytesIO(image_data)).convert("RGB")
        image_np = np.array(image)

        # Process the image with YOLOv11
        processed_image_np, bounding_boxes, class_names, confidences = process_image(image_np, prompt)

        # Convert processed image to base64 for JSON response
        processed_image_pil = Image.fromarray(processed_image_np)
        buffer = BytesIO()
        processed_image_pil.save(buffer, format="PNG")
        img_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

        print("Image processed successfully.")


        # Prepare response data
        response_data = {
            "processed_image": f"data:image/png;base64,{img_base64}",
            "bounding_boxes": bounding_boxes,  # [[x1, y1, x2, y2], ...]
            "classes": class_names,           # ["class_name1", "class_name2", ...]
            "confidences": confidences,       # [conf1, conf2, ...]
            "message": "Image processed successfully"
        }

        return JSONResponse(content=response_data, status_code=200)

    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 185MB/s]


In [3]:

# Function to start ngrok and FastAPI server
async def start_server(Token_ngrok):
    # Set your ngrok authtoken
    ngrok.set_auth_token(Token_ngrok)  # Replace with your ngrok authtoken

    # Start ngrok tunnel
    public_url = ngrok.connect(8000, bind_tls=True).public_url
    print(f"ngrok public URL: {public_url}")

    # Configure and run the FastAPI server
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)
    await server.serve()



In [ ]:

Token_ngrok = "2WPdKpWKTAPyjgMZJ0Wzuzz201h_7ygriY2pbSbNbx8VboPoj"  # Replace with your actual ngrok token
# Run the server
asyncio.run(start_server(Token_ngrok))

ngrok public URL: https://8f73-34-87-145-147.ngrok-free.app


INFO:     Started server process [840]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2001:fb1:c1:c843:555d:cf2e:13a4:7e3f:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2001:fb1:c1:c843:555d:cf2e:13a4:7e3f:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
